In [7]:
###############################
# Import SDK & Libraries
###############################

import prisma_sase
import time

In [24]:
###############################
# Service Account Details
###############################

client_id = "paste client ID"
client_secret = "paste client secret"
tsg_id = "paste TSG ID"

In [3]:
##############################################################
# Instantiate SDK & login using service account
##############################################################

sase_session = prisma_sase.API()
sase_session.interactive.login_secret(client_id=client_id, client_secret=client_secret, tsg_id=tsg_id)

	Latest Version: 6.3.2b1
	Current Version: 6.3.1b1
	For more info, see 'https://github.com/PaloAltoNetworks/prisma-sase-sdk-python'. Additionally, this message can be suppressed by instantiating the API with API(update_check=False).



True

In [6]:
###############################
# Create Site Dictionary
###############################

site_id_name = {}
site_name_id = {}

#
# Site ID to be used for template creation
#
SITE_ID = None

resp = sase_session.get.sites()
if resp.cgx_status:
    sitelist = resp.cgx_content.get("items", None)
    for site in sitelist:
        site_id_name[site["id"]] = site["name"]
        site_name_id[site["name"]] = site["id"]
        
        if site["name"] == "Branch 1 - Raleigh":
            SITE_ID = site["id"]
            
else:
    print("ERR: Could not retrieve Sites")
    prisma_sase.jd_detailed(resp)
        
        
if SITE_ID is None:
    print("ERR: Site not found!!")

In [9]:
###############################
# Create Site Template
###############################

TEMPLATE_ID = None
data = {
    "site_id":SITE_ID,
    "site_type":"SPOKE",
    "template_name":"template_name",
    "template_description":"template_description"
}

resp = sase_session.post.bulkconfigurations_sitetemplates(data=data)
if resp.cgx_status:
    TEMPLATE_ID = resp.cgx_content.get("id", None)
    print("Site Template Create Request Sent. Site Template ID: {}".format(TEMPLATE_ID))
else:
    print("ERR: Could not send Site Template Create Request")
    prisma_sase.jd_detailed(resp)

Site Template Create Request Sent. Site Template ID: 1715361469108000196


In [10]:
###############################
# Check Site Template Status
###############################

created = False
while not created:
    resp = sase_session.get.bulkconfigurations_sitetemplates_status(sitetemplate_id=TEMPLATE_ID)
    if resp.cgx_status:
        status = resp.cgx_content.get("status", None)
        status_desc = resp.cgx_content.get("status_description", None)
        
        print("{} [{}]".format(status, status_desc))
        if status == "SUCCESS":
            created = True
        else:
            print("Sleeping 2 seconds..")
            time.sleep(2)
        
            
    else:
        print("ERR: Could not retrieve status")
        prisma_sase.jd_detailed(resp)
        print("Sleeping 2 seconds..")
        time.sleep(2)

INIT [INIT]
Sleeping 2 seconds..
INIT [INIT]
Sleeping 2 seconds..
INIT [INIT]
Sleeping 2 seconds..
SUCCESS [Template created successfully.]


In [14]:
##############################################
# GET Sites Template & Variable Map
##############################################
resp = sase_session.get.bulkconfigurations_sitetemplates(sitetemplate_id=TEMPLATE_ID)
if resp.cgx_status:
    variable_map = resp.cgx_content.get("variable_map", None)
    print("Variable Map: ")
    prisma_sase.jd_detailed(variable_map)
                                       

Variable Map: 
{
    "site_name": "Branch 1 - Raleigh",
    "street": "316 Faynetteville St.",
    "ion_sw_version": "6.2.2-b1",
    "ion_name_1": "device-1",
    "ion_serial_number_1": "4229b40f-2a39-7947-3a0b-3a6e99591238"
}


In [22]:
###############################################
# Deploy using Site Template
###############################################
data = {
    "variable_map":[
        {
            "site_name":"test-tk1",
            "street":"123 main st",
            "ion_sw_version":"6.2.2-b1",
            "ion_name_1":"test-tk1-ion",
            "ion_serial_number_1":""
        },
        {
            "site_name":"test-tk2",
            "street":"123 main st",
            "ion_sw_version":"6.2.2-b1",
            "ion_name_1":"test-tk2-ion",
            "ion_serial_number_1":""
        },
        {
            "site_name":"test-tk3",
            "street":"123 main st",
            "ion_sw_version":"6.2.2-b1",
            "ion_name_1":"test-tk3-ion",
            "ion_serial_number_1":""
        }
    ]
}

JOB_IDS = []
jobid_sitename = {}
resp = sase_session.post.bulkconfigurations_sitetemplates_deployments(sitetemplate_id=TEMPLATE_ID, data=data)
if resp.cgx_status:
    deployments = resp.cgx_content.get("items", None)
    for deployjob in deployments:
        sitename = deployjob.get("variable_map", None)["site_name"]
        jobid = deployjob.get("deployment_id", None)
        jobid_sitename[jobid] = sitename
        
        print("Job created for {} [{}]".format(sitename, jobid))
        JOB_IDS.append(jobid)

        
else:
    print("ERR: Could not create deployment jobs")
    prisma_sase.jd_detailed(resp)

Job created for test-tk1 [1715364855762018096]
Job created for test-tk2 [1715364855870018596]
Job created for test-tk3 [1715364855973019096]


In [23]:
##############################################################
# Get Deployment Status using deployment ID
##############################################################

for jobid in JOB_IDS:
    print("**************************")
    print("Site: {} Job: {}".format(jobid_sitename[jobid], jobid))

    status = "INIT"
    while status != "SUCCESS":
        url = "https://api.sase.paloaltonetworks.com/sdwan/v2.0/api/bulkconfigurations/sitetemplates/{}/deployments/{}/status".format(TEMPLATE_ID, jobid)
        resp = sase_session.rest_call(method="GET", url=url)
        if resp.cgx_status:
            status = resp.cgx_content.get("status", None)
            status_description = resp.cgx_content.get("status_description", None)
            print("\tStatus:{} [{}]".format(status, status_description))
            
            if status != "SUCCESS":
                print("\tDeployment ongoing. Sleeping 2 seconds..")
                time.sleep(2)
                
            else:
                print("\tDeployment Complete!!")
                

        else:
            print("ERR: Could not retrieve deployment status")
            prisma_sase.jd_detailed(resp)

**************************
Site: test-tk1 Job: 1715364855762018096
	Status:INPROGRESS [INPROGRESS]
	Deployment ongoing. Sleeping 2 seconds..
	Status:INPROGRESS [INPROGRESS]
	Deployment ongoing. Sleeping 2 seconds..
	Status:INPROGRESS [INPROGRESS]
	Deployment ongoing. Sleeping 2 seconds..
	Status:SUCCESS [Successfully deployed site 'test-tk1]
	Deployment Complete!!
**************************
Site: test-tk2 Job: 1715364855870018596
	Status:INPROGRESS [INPROGRESS]
	Deployment ongoing. Sleeping 2 seconds..
	Status:INPROGRESS [INPROGRESS]
	Deployment ongoing. Sleeping 2 seconds..
	Status:INPROGRESS [INPROGRESS]
	Deployment ongoing. Sleeping 2 seconds..
	Status:INPROGRESS [INPROGRESS]
	Deployment ongoing. Sleeping 2 seconds..
	Status:SUCCESS [Successfully deployed site 'test-tk2]
	Deployment Complete!!
**************************
Site: test-tk3 Job: 1715364855973019096
	Status:INIT [INIT]
	Deployment ongoing. Sleeping 2 seconds..
	Status:INPROGRESS [INPROGRESS]
	Deployment ongoing. Sleeping 2

In [25]:
##############################################################
# Query API for all Deployments
##############################################################

data = {
    "query_params":{},
    "limit":10,
    "dest_page":1,
    "sort_params":{"_updated_on_utc":"desc"},
    "retrieved_fields":[],
    "retrieved_fields_mask":True
}


resp = sase_session.rest_call(method="POST", 
                     data=data,
                     url="https://api.sase.paloaltonetworks.com/sdwan/v2.0/api/bulkconfigurations/sitetemplates/deployments/query")
prisma_sase.jd_detailed(resp.cgx_content)


{
    "_etag": 0,
    "_content_length": "0",
    "_schema": 0,
    "_created_on_utc": 0,
    "_updated_on_utc": 0,
    "_status_code": "200",
    "_request_id": "1715364959016017199995733041024699950510",
    "count": 10,
    "items": [
        {
            "_content_length": 0,
            "_created_on_utc": 17153648558700185,
            "_updated_on_utc": 17153648939710052,
            "variable_map": {
                "site_name": "test-tk2",
                "street": "123 main st",
                "ion_sw_version": "6.2.2-b1",
                "ion_name_1": "test-tk2-ion",
                "ion_serial_number_1": ""
            },
            "deployment_id": "1715364855870018596",
            "template_id": "1715361469108000196",
            "site_id": "1715364867888006196",
            "site_name": "test-tk2",
            "status": "SUCCESS",
            "status_description": "Successfully deployed site 'test-tk2",
            "template_name": "template_name",
            "_creat